<center>

<h1 style="text-align:center"> Polymorphic Lambda Calculus: System F </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review

### Previously

* Simply Typed Lambda Calulus.
  + Products, Sums, Type Soundness
  + Curry Howard Correspondence
  + Type Erasure

### Today

* System F: Polymorphic Lambda Calculus

$
\newcommand{\stlc}{\lambda^{\rightarrow}}
\require{color}
\newcommand{\c}[2]{{\color{#1}{\text{#2}}}}
$

## Doubling functions

In simply typed lambda calculus, the `twice f x = f (f x)` function must be specified at every type:

$
twice\_unit = \lambda f:1 \rightarrow 1.\lambda x:1. f~(f ~x) \\
twice\_sum = \lambda f: A+B \rightarrow A+B.\lambda x:A+B. f~(f ~x) \\
twice\_a2a = \lambda f: (A \rightarrow A) \rightarrow (A \rightarrow A). \lambda x:A \rightarrow A. f ~(f ~x)
$

Clearly this copy-pasting of code and changing the type violates the basic dictum of software engineering:

**Abstraction Principle:** Each significant piece of functionality in a program should be implemented in just one place in the source code.

## Universal Type

* System F (polymorphic lambda calculus) is obtained by extending Curry Howard isomorphism to $\forall$.
* System F has dedicated syntax for type and term families such as `twice`:
  + The System F term $\Lambda \alpha.\lambda f: \alpha \rightarrow \alpha.\lambda x:\alpha.f ~(f ~x))$ has type $\forall \alpha.(\alpha \rightarrow \alpha) \rightarrow \alpha \rightarrow \alpha$.
* $\Lambda \alpha.M$ is called a **type abstraction**.
  + Correspondingly, we also have a **type application** of the form $M A$, where $M$ is a term and $A$ is a type.
* System F offers polymorphism
  + $\c{red}{But which kind?}$

## Polymorphism

Polymorphism comes in various kinds

1. Ad-hoc polymorphism: 
   * function overloading (C++,Java)
   * operator overloading (C++,Java,Standard ML, C)
   * typeclasses (Haskell)
2. Subtyping:
   * subclasses (C++, Java, OCaml)
   * Row polymorphism (OCaml)
3. Parametric Polymorphism:
   * polymorphic data and functions in OCaml, Haskell, Standard ML.
   * Generics in Java, C#

## Polymorphism

* Unqualified term "polymorphism" means different things depending on who you talk to
  + OO person: subtype polymorphism; parametric polymorphism is generics
  + FP person: parametric polymorphism.

## Types

Types in System F are as follows:

\\[
\begin{array}{rcll}
\text{Types: } A,B & ::=  & \alpha & \text{(type variable)} \\
                   & \mid & A \rightarrow B & \text{(function type)} \\
                   & \mid & \forall \alpha.A & \text{(universal type)}
\end{array}
\\]

* We have dropped pairs, sums, 1 (unit), 0 types from $\lambda^{\rightarrow}$
  + Can be encoded!
* Notice the symmetry to untyped lambda calculus terms.

## Free Type Variables

We define free type variables on System F types similar to free variables on lambda terms.

\\[
\begin{array}{rcl}
FTV(\alpha) & = & \alpha \\
FTV(A \rightarrow B) & = & FTV(A) \cup FTV(B) \\
FTV(\forall \alpha.A) & = & FTV(A) \setminus \{\alpha\}
\end{array}
\\]

## Terms

\\[
\begin{array}{rcll}
\text{Terms: } M,N & ::=  & x & \text{(variable)} \\
                   & \mid & M~N & \text{(application)} \\
                   & \mid & M~[A] & \text{(type application)} \\
                   & \mid & \lambda x:A.M & \text{(abstraction)} \\
                   & \mid & \Lambda \alpha.M & \text{(type abstraction)}
\end{array}
\\]

$\newcommand{\inferrule}[2]{\displaystyle{\frac{#1}{#2}}}$

## Substitution

### For terms

If $M$ is a term, $N$ a term, and $x$ a variable, we write $M[N/x]$ for capture-free substitution of $N$ for $x$ in $M$.

### For types

If $M$ is a term, $B$ a type, and $\alpha$ a type variable, we write $M[B/\alpha]$ for capture-free substitution of $B$ for $\alpha$ in $M$.


## Reduction: Boring Rules

\\[
\begin{array}{ccc}
\inferrule{M \rightarrow M'}{M~N \rightarrow M'~N} & 
\inferrule{N \rightarrow N'}{M~N \rightarrow M~N'} &
\inferrule{M \rightarrow M'}{\lambda x.M \rightarrow \lambda x.M'}
\end{array}
\\]

\\[
\begin{array}{cc}
\inferrule{M \rightarrow M'}{M~A \rightarrow M'~A} &
\inferrule{M \rightarrow M'}{\Lambda \alpha.M \rightarrow \Lambda \alpha.M'}
\end{array}
\\]

## Reductions: Interesting Rules

\\[
\begin{array}{rcl}
(\lambda x:A.M) ~N       & \rightarrow & M[N/x] \\
(\Lambda \alpha.M) ~A    & \rightarrow & M[A/\alpha] \\
\lambda x:A.M ~x         & \rightarrow & \lambda M \quad \text{, if } x \notin FV(M) \\
\lambda \alpha.M ~\alpha & \rightarrow & \lambda M \quad \text{, if } \alpha \notin FTV(M)
\end{array}
\\]

## Encodings : Booleans

In untyped lambda calculus, `tru` and `fls` values were:

```ocaml
tru = 𝜆t.𝜆f.t
fls = 𝜆t.𝜆f.f
```

In simply typed lambda calculus, there was a `tru` and `fls` for each type:

```ocaml
tru_int = 𝜆t:int.𝜆f:int.t
tru_bool = 𝜆t:float.𝜆f:float.t
```

## Encoding Booleans

In System F, there is a single polymorphic `tru` and `fls` values of type `bool`. We define the type `bool` and the booleans as

\\[
\begin{array}{rcl}
bool & = & \forall \alpha. \alpha \rightarrow \alpha \rightarrow \alpha \\
tru  & = & \Lambda \alpha. \lambda t:\alpha. \lambda f:\alpha. t \\
fls  & = & \Lambda \alpha. \lambda t:\alpha. \lambda f:\alpha. f \\
\end{array}
\\]

Easy to see that the judgements $\vdash tru : bool$ and $\vdash fls : bool$ hold.

## Encoding boolean operations

`test` function is defined as:

\\[
\begin{array}{rcl}
test : \forall \alpha. bool \rightarrow \alpha \rightarrow \alpha \rightarrow \alpha \\
test = \Lambda \alpha. \lambda b : bool. b~\alpha
\end{array}
\\]

Notice the application of $b$ to $\alpha$ above. 

We can define logical operators as follows:

\\[
\begin{array}{rcl}
and & = & \lambda x:bool.\lambda y:bool.x ~bool ~y ~fls \\
or  & = & \lambda x:bool.\lambda y:bool.x ~bool ~tru ~y \\
not & = & \lambda x:bool.x ~bool ~fls ~tru 
\end{array}
\\]

## Encoding natural numbers